In [1]:
import moviepy.editor as mpe
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
import sounddevice as sd
from PIL import Image

In [ ]:
# Gravar um video de si mesmo lendo o texto no arquivo anexo 
video = mpe.VideoFileClip("WIN_20241202_13_46_54_Pro.mp4")
video.ipython_display(width=480, maxduration=300)

In [ ]:
#nverter a imagem, no eixo, a cada 20 segundo (escolhi alterar o eixo y)
def espelha_em_y(pegar_frame, tempo):
    frame = pegar_frame(tempo)

    # Logica para inverter o video no eixo y
    if int(tempo // 20) % 2 == 0:
        return frame[::-1, :]
    else:
        return frame
#Apos fazer a funcao, irei aplica-la em todo o video, e depois mostrar o video, conforme pede o ex
video_em_relacao_y = video.fl(espelha_em_y)
video_em_relacao_y.ipython_display(width=480, maxduration=400)

In [ ]:
# Diminiur o som, de forma gradativa, a cada 30 segundos, de forma que os últimos 10 segundos fique em silêncio (sem som)
#funcao para reduzir o som
def reducao_ate_sem_som(pegar_frame, tempo):
    
    # para  ter a duracao total do video
    duracao_video = video.duration

    #Aqui estou pegando o valor maximo do video, que e o 100% ou 1
    fator_de_duracao = np.ones_like(tempo)

    #Apos varias pesquisas, vi que o video pode ser considerado como um vetor de imagens
    #Com isso, passar o parametro tempo de forma direta, acarretaria em erro, por possuir apenas um valor
    #Sendo assim, a melhor decisao foi: para cada condicao, eu atribuiria uma mascara

    #Primeira mascara
    mascara1 = tempo < 150
    fator_de_duracao[mascara1] = 1 - (tempo[mascara1] // 30) * 0.2 #Reduz gradualmente o volume: a cada 30 segundos, diminui o volume em 20% 

    #Segunda mascara
    mascara2 = tempo > 150
    fator_de_duracao[mascara2] = np.maximum(0, 1 - (tempo[mascara2] - 150) / 10) #O volume é reduzido linearmente conforme o tempo aumenta além de 150 segundos.
    
    return pegar_frame(tempo) * fator_de_duracao[:, np.newaxis]

#Feito isso, vou varrer todo o video com a funcao fl, substituir o audio original e salvar o audio
    
audio_definido = video.audio.set_fps(44100).fl(reducao_ate_sem_som)

video_definido_novo = video.set_audio(audio_definido)

video_definido_novo.write_videofile("video_editado.mp4", codec="libx264", audio_codec="aac")

In [ ]:
#Após o primeiro minuto, 60 segundos, fazer um corte para o segundo 80 
#  Por fim, insira os 20 segundos cortados acima no fim do video 
#Primeiramente, dividi o video em 3 etapas:
# De (0 a 60), de (60,80) e depois ate 80 que e ate o final do video  (dividindo em subclips)

_videoc1 = video.subclip(0,60)
_videoc2 = video.subclip(60,80)
_videoc3 = video.subclip(80)

#Aqui, usei a funcao de concatenacao para juntar os 3 videos, na ordem estimulada pelo trabalho
#E por fim, carreguei o video e depois salvei como "video_carregado"

_videoc4 = mpe.concatenate_videoclips([_videoc1, _videoc3, _videoc2])
_videoc4.ipython_display(width=480, maxduration=600) #Exibe o vídeo concatenado
_videoc4.write_videofile("video_carregado_shrek.mp4",codec="libx264", audio_codec="aac" )